In [ ]:
!pip install opencv-python mediapipe

import cv2
import mediapipe as mp
import numpy as np
from google.colab import drive

drive.mount('/content/drive')

video_path = '/content/drive/MyDrive/Source/No kidnap.mp4'

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5)

mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)

def detect_kidnapping(image):
    results_face = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    results_holistic = holistic.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    kidnapping_risk = 0
    annotations = []

    if results_face.multi_face_landmarks:
        for face_landmarks in results_face.multi_face_landmarks:

            kidnapping_risk += 1
            annotations.append(("Face Detected", (0, 0), (0, 255, 0)))


    if results_holistic.left_hand_landmarks or results_holistic.right_hand_landmarks:
        annotations.append(("Hand Gestures Detected", (0, 50), (0, 255, 255)))
        kidnapping_risk += 2
    if results_holistic.pose_landmarks:
      annotations.append(("Body Pose Detected", (0, 100), (255, 255, 0)))
      kidnapping_risk += 0

    for text, pos, color in annotations:
      cv2.putText(image, text, pos, cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    return image, kidnapping_risk
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print(f"Error opening video file: {video_path}")
else:
  ret, frame = cap.read()
  annotated_frame, risk_level = detect_kidnapping(frame)
  cv2.imwrite('/content/kidnapping_analysis.jpg', annotated_frame)
  print(f"Kidnapping risk level: {risk_level}")
  cap.release()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Kidnapping risk level: 0
